In [11]:
from typing import Literal

In [12]:
import torch 
import numpy as np 
from transformers import AutoProcessor, AutoModel, BarkProcessor, BarkModel 


/home/ujjwal/learning/fastapi_for_ml_ai_enginners /.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
device= "cuda"
processor= AutoProcessor.from_pretrained("suno/bark-small", device= device)

In [ ]:
model= AutoModel.from_pretrained("suno/bark-small")

## generate audio

In [15]:

VoicePresets = Literal["v2/en_speaker_1", "v2/en_speaker_9"] 
prompt = "hellow my name is ujjwal " 
preset:VoicePresets = 'v2/en_speaker_1'

inputs= processor(text=[prompt], return_tensors= 'pt', voice_preset=preset )

In [16]:
inputs ## tensor

{'input_ids': tensor([[ 61694,  16602,  15127,  11324,  10124,    189,  10418, 102893,  10161,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              

In [17]:
inputs["attention_mask"]

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

## generate an audio array that contains amplitude values of the synthesized audio
signal over time.

In [18]:
## outputs 
output= model.generate(**inputs , do_sample=True).cpu().numpy().squeeze() 
output

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


array([0.00093562, 0.00053802, 0.00022066, ..., 0.00036183, 0.00049017,
       0.00059302], shape=(98880,), dtype=float32)

## Get the sampling rate from model generating configurations, which can be used
to produce the audio.

In [19]:
# sample rate  - 1 second audio = 24,000 waveform points
## needs this parameter to save on librosa or soundfile  
sample_rate= model.generation_config.sample_rate
sample_rate

24000

When you generate audio using a model, the output is a sequence of floating-point
numbers that represent the amplitude (or strength) of the audio signal at each point
in time.

To play back this audio, it needs to be converted to a digital format that can be sent to
the speakers. that is where soundfile library helps

Sampling rate: 


        fs​=100 Hz⇒T=1001​=0.01s 


So measurements happen at: 

                t=0, 0.01, 0.02, 0.03,… 

        

(FOR EVERY even distance of 0.01 we have the value of sound this means we dont hve value of sound in 0.002 , 0.003, 0.009 but we have of 0.01 ,  0.02 but not 0.011 ...)




In [22]:
#so more hrz more better = more sample more better 

In [ ]:
## audio array to buffer 
from io import BytesIO ## bytesIO is in memoery file 
import soundfile 

buffer= BytesIO()
soundfile.write(buffer, output, sample_rate, format='wav')



## 1.  `BytesIO()` 

`BytesIO` is an **in-memory file**.

Just like a real file on disk, it has:

- raw byte content  
- a cursor (pointer) that tracks where the next read or write happens  

You can visualize it like this:



*************

[ byte0 | byte1 | byte2 | ... | byteN ]



^
pointer

***********************


The pointer decides **where reading or writing starts**.

---

## 2. What Happens During `soundfile.write(...)`

```python
soundfile.write(buffer, audio_array, sample_rate, format="wav")


What this does internally:

        - WAV bytes are written starting at byte 0

        - As bytes are written, the pointer keeps moving forward

        - When writing finishes, the pointer ends up at the end of the buffer

In [ ]:
## audio array to buffer 
from io import BytesIO ## bytesIO is in memoery file 
import soundfile 


buffer= BytesIO()
soundfile.write(buffer, output, sample_rate, format='wav')
buffer.seek(0)
with open("output.wav", "wb") as f:
    buffer.seek(0)
    f.write(buffer.read())

In [41]:
import IPython.display as ipd
buffer.seek(0)  # read from start
ipd.Audio(buffer.read(), rate=sample_rate)